In [ ]:
import numpy as np
import pandas as pd


from keras.models import Sequential
from keras.layers import Embedding, LSTM, GRU, Dense
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [2]:
path = 'your_path_here.csv' #https://www.kaggle.com/datasets/andrezaza/clapper-massive-rotten-tomatoes-movies-and-reviews

data_set = pd.read_csv(path)

y= data_set.iloc[:,0].values
X = data_set.iloc[:,1].values

In [3]:
from sklearn.model_selection import train_test_split
(X_train_text,X_test_text,y_train,y_test) = train_test_split(X,y, test_size = 0.3, random_state =  2020)

In [4]:
tokenizer = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(X)

In [5]:
X_train_token  = tokenizer.texts_to_sequences(X_train_text)
X_test_token = tokenizer.texts_to_sequences(X_test_text)

In [6]:
token_string = tokenizer.word_index
Inverse_map = dict(zip(token_string.values(), token_string.keys()))

In [7]:
def token_to_text(tokens):
    strings = [Inverse_map[token] for token in tokens if token != 0]
    text = " ".join(strings)
    return text

In [8]:
len_max = 241
#Padding the sequence 
X_train_pad = pad_sequences(X_train_token, maxlen = len_max, padding = 'pre')
X_test_pad = pad_sequences(X_test_token, maxlen = len_max, padding ='pre')


In [9]:
print(X_train_pad[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [10]:
#Model
embedding_size = 20
Model = Sequential()
Model.add(Embedding(input_dim = 20000, input_length = len_max, output_dim = embedding_size, name = 'Embedding_Layer'))
Model.add(GRU(units = 16, return_sequences = True, name='First_Layer'))
Model.add(GRU(units = 8, return_sequences = True, name ='Second_layer'))
Model.add(GRU(units = 4, name = "Third_Layer"))
Model.add(Dense(1, activation = 'sigmoid', name ="Dense_Layer"))
optimizer = Adam(lr = 0.001)
Model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'], optimizer = optimizer)
Model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 241, 20)           400000    
_________________________________________________________________
First_Layer (GRU)            (None, 241, 16)           1824      
_________________________________________________________________
Second_layer (GRU)           (None, 241, 8)            624       
_________________________________________________________________
Third_Layer (GRU)            (None, 4)                 168       
_________________________________________________________________
Dense_Layer (Dense)          (None, 1)                 5         
Total params: 402,621
Trainable params: 402,621
Non-trainable params: 0
_________________________________________________________________


In [11]:
#Fitting the model
Model.fit(X_train_pad, y_train, validation_split = 0.05,epochs =5, batch_size = 32)
Model.summary()

Epoch 1/5
9975/9975 [==============================] - 2036s 204ms/step - loss: 0.4304 - accuracy: 0.7986 - val_loss: 0.3871 - val_accuracy: 0.8254
Epoch 2/5
9975/9975 [==============================] - 18231s 2s/step - loss: 0.3529 - accuracy: 0.8456 - val_loss: 0.3680 - val_accuracy: 0.8379
Epoch 3/5
9975/9975 [==============================] - 106802s 11s/step - loss: 0.3145 - accuracy: 0.8665 - val_loss: 0.3570 - val_accuracy: 0.8457
Epoch 4/5
9975/9975 [==============================] - 2067s 207ms/step - loss: 0.2820 - accuracy: 0.8832 - val_loss: 0.3623 - val_accuracy: 0.8471
Epoch 5/5
9975/9975 [==============================] - 24245s 2s/step - loss: 0.2535 - accuracy: 0.8978 - val_loss: 0.3602 - val_accuracy: 0.8537
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 241, 20)           400000    
______________________________________________

In [12]:
result = Model.evaluate(X_train_pad,y_train)
Evaluate = Model.evaluate(X_test_pad,y_test)
print("The accuracy on the training set is {0:.2%}".format(result[1]))
print("The accuracy on the test set is {0:.2%}".format(Evaluate[1]))

4500/4500 [==============================] - 252s 56ms/step - loss: 0.3533 - accuracy: 0.8545
The accuracy on the training set is 91.87%
The accuracy on the test set is 85.45%


In [13]:
def classifier(text):
    text_seq = tokenizer.texts_to_sequences(text)
    #text_seq = [np.array(a) for a in text_seq]
    text_token = pad_sequences(text_seq, maxlen = len_max, padding = 'pre')
    pred = Model.predict(text_token)
    if pred[0] >= 0.5:
        result = 'Positive Review! Thank you'
    else:
        result = "Negative Review! We will improve our delivery"
    return result

In [14]:
text = ["I  love the movie"]
print(classifier(text))

Negative Review! We will improve our delivery


In [15]:
#Saving the Model
Model.save("model/Model.h5")

In [16]:
import pickle 
with open('tokenize.pickle', 'wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)